In [1]:
import pandas as pd
import numpy as np

## Класс функций потерь

In [2]:
class LossFunction:
    """
    Класс содержащий классы функций ошибок,
    которые содержат в себе методы вычисляющие потерю и производные.
    
    Каждый метод классов принимает на вход два массива:
    y_true: np.array содержащий настоящии значения
    y_pred: np.array содержащий предсказанные значения
    
    Реализованны слудующие функции потерь:
    MSE: mean squeared error
    MAE: mean absolute error
    RMSE: roof mean squear error
    MAPE: mean absolute percentage error
    Huber: huber function
    LogCosh: logarithm of the hyperbolic cosine error
    Log: logarithm error
    CosineSimilarity: cosine similarity
    """
    
    class Mse:
        
        def loss(self, y_true, y_pred):
            return ((y_true - y_pred) ** 2).mean()
        
        def deriv(self, y_true, y_pred):
            return -2 * (y_true - y_pred)
        
    class Mae:
    
        def loss(self, y_true, y_pred):
            return (np.abs(y_true - y_pred)).mean()

        def deriv(self, y_true, y_pred):
            return (y_pred - y_true)/np.abs(y_pred - y_true)
        
    class Rmse(Mse):

        def loss(self, y_true, y_pred):
            return np.sqrt(((y_true - y_pred) ** 2).mean())

        def deriv(self, y_true, y_pred):
            return (1/(2 * Mse.loss(self, y_true[i], y_pred[i]))) * Mse.deriv(self, y_true[i], y_pred[i])
        
    class Mape:
    
        def loss(self, y_true, y_pred):
            return 100 * (np.abs((y_true - y_pred) / (y_true + np.exp(-20))).mean())

        def deriv(self, y_true, y_pred):
            return (y_pred - y_true)/(np.abs(y_true + np.exp(-20)) * np.abs(y_pred - y_true))
        
    class Huber(Mse, Mae):
    
        def loss(self, y_true, y_pred):
            losses = []
            for i in range(y_true.size):
                if (y_true[i] - y_pred[i]) <= 1:
                    losses.append(Mse.loss(self, y_true[i], y_pred[i]))
                else: losses.append(Mae.loss(self, y_true[i], y_pred[i]))
            return np.array(losses).mean()

        def deriv(self, y_true, y_pred):
            deriv = []
            for i in range(y_true.size):
                if (y_true[i] - y_pred[i]) <= 1:
                    deriv.append(Mse.deriv(self, y_true[i], y_pred[i]))
                else: deriv.append(Mae.deriv(self, y_true[i], y_pred[i]))
            return np.array(deriv)
        
    class LogCosh:
    
        def loss(self, y_true, y_pred):
            return np.log((np.exp(y_true - y_pred) + np.exp(y_pred - y_true))/2).sum()

        def deriv(self, y_true, y_pred):
            return (np.exp(2 * (y_pred)) - np.exp(2 * (y_true))) / (np.exp(2 * (y_pred)) + np.exp(2 * (y_true)))
        
    class Log:
    
        def loss(self, y_true, y_pred):
            return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).sum()

        def deriv(self, y_true, y_pred):
            return -((y_pred - y_true) / (y_pred**2 - y_pred))
        
    class CosineSimilarity:
        
        def loss(self, y_true, y_pred):
            return (y_true*y_pred/np.maximum(np.sqrt((y_true**2).sum()*(y_pred**2).sum()), np.exp(-20))).mean()
        
        def deriv(self, y_true, y_pred):
            const = np.maximum(np.sqrt((y_true**2).sum()*(y_pred**2).sum()), np.exp(-20))
            return y_pred/const-self.loss(y_true, y_pred)/np.maximum((y_true**2).sum(), np.exp(-20))


## Класс функций активации

In [3]:
# add parametrs
class Activation:
    """
    Класс содержащий классы функций активации,
    которые содержат в себе методы вычисляющие активацию и производные.
    
    Каждый метод классов принимает на вход число:
    x: int
        выходное значение нейрона
    
    Реализованны слудующие функции активации:
    Sigmoid,
    Relu,
    Leakly_relu.
    """
    
    class Sigmoid:
        def active(self, x):
            return 1 / (1 + np.exp(-x))
        
        def deriv(self, x):
            x = self.active(x)
            return x * (1 - x)
        
        
    class Relu:
        def active(self, x):
            return np.maximum(0, x)
        
        def deriv(self, x):
            return np.maximum(0, x)
        
        
    class Leakly_relu:
        def active(self, x):
            return np.maximum(0.1 * x, x)
        
        def deriv(self, x):
            return np.maximum(0.1 * x, x)
    

## Класс нейрона

In [4]:
class Neuron:
    """
    Один нейрон, объявление класса происходит внутри класс Linear,
    начальный вес и смещение инициализируются рандомно
    """
    
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.sum_neuron = 0
        
    def forward(self, input_vector):
        self.sum_neuron = (np.dot(self.weights, input_vector) + self.bias).ravel()
        return self.sum_neuron
    

## Класс нейроного слоя

In [5]:
class Linear:
    """
    Один слой нейронной сети
    на вход принимаются значения
    input_size: int
        число передаваемых связей
    output_size: int
        число нейронов в слое
    """
    
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.neurons = []  # хронит нейроны
        
        for layer in range(output_size):
            np.random.seed(21) 
            self.weights = np.random.rand(1, input_size)  # инициализация веса
            np.random.seed(21) 
            self.bias = np.random.randn()  # инициализация смещения
            self.neurons.append(Neuron(self.weights, self.bias))
# 
            
    def forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            n = neuron.forward(inputs).ravel()
            outputs.append(n)
        return np.array(outputs).ravel()


## Класс с архетектурой сети

In [6]:
class Network:
    """Создание нейронной сети"""
    def __init__(self):
        self.activation_layer = Activation.Sigmoid()
        self.linear1 = Linear(2, 2)
        self.linear2 = Linear(2, 1)
        self.layers = [self.linear2, self.linear1]  # массив слоев для обратного хода
        self.out = []
        
    def forward(self, inputs):
        l1 = self.linear1.forward(inputs)
        activ_l1 = self.activation_layer.active(l1)
        l2 = self.linear2.forward(activ_l1)
        result = self.activation_layer.active(l2)
        self.out = [result, activ_l1]  # массив выходов для обратного хода
        return result
        

## Реализация обратного распростронения ошибки

In [7]:
def back_prop(y_true, y_pred, net, inputs, loss, lr): 
    """
    На вход принимаются значения:
    y_true: list
        содержит таргет
        
    y_pred: list
        содержит предсказание
        
    net: obj Network
        объект нашей архетектуры нейронной сети
        
    inputs: list
        входные значения
        
    loss: obj LossFunction
        функция потерь
        
    lr: float
        скорость обучения
    """        
    ypred = loss.deriv(y_true, y_pred)  # производная от функции потери
    h = []  # FIFO хранящий значения нейронов
    # выход каждого слоя плюс входные значения и единица для удобства
    ls = net.out
    ls.append(inputs)
    ls.append(1)
    
    layers = net.layers  # массив слоев нейронной сети
    for layer in layers:
        bias_temp = []  # массив хранящий временные свободные коэффициенты нейрона
        for neuron in layer.neurons:
            deriv = net.activation_layer.deriv(neuron.sum_neuron)[0]  # производная нейрона
            neuron.bias = [(ypred * lr * deriv)]
            
            weights_temp = []
            count = 0 
            if layer != layers[0]:
                hh = h.pop()
            else: hh = 1
            for weight in neuron.weights[0]:
                
                h_temp = ls[layers.index(layer)+1][count]  #h1
                weight_temp = h_temp * deriv #w5
                
                h.append(weight_temp * deriv)  #h1_new
                weights_temp.append((weight - (ypred* lr * hh * weight_temp)))

                count+=1
            weights_temp = np.array(weights_temp).reshape(1, 2)
            neuron.weights = weights_temp               

In [8]:
net = Network()
learn_rate = 0.1
epochs = 100
accur  = LossFunction.Mae()
loss_f = LossFunction.Mse()
y_trues = []
def train(data, all_y_trues = None):
        
        for epoch in range(epochs):
            y_preds = []
            for x, y_true in zip(data, all_y_trues):   
                y_pred = net.forward(x)
                back_prop(y_true, y_pred, net, x, loss_f, learn_rate)
                y_preds.append(y_pred[0])

          # --- Считаем полные потери в конце каждой эпохи
            if epoch % 10 == 0:
                accurs = accur.loss(np.array(all_y_trues), y_preds)
                losses = loss_f.loss(np.array(all_y_trues), y_preds)
                print("Epoch %d accur: %.3f, loss: %.3f" % (epoch, accurs, losses))
                
# функция для тестового ввода данных               
def test(data):
#     for x in data:
    y_pred = net.forward(data)
    return y_pred[0]

In [9]:
# Определим набор данных
# Данные включают в себя два параметра: рост, вес ребенка
# Показатели смещены на 100 см и 20 кг
data = np.array([
  [-2, -1],  # Алиса
  [25, 6],   # Боб
  [17, 4],   # Чарли
  [-15, -6], # Диана
])
all_y_trues = np.array([
  1, # Алиса
  0, # Боб
  0, # Чарли
  1, # Диана
])

# Обучаем нашу нейронную сеть!
train(data, all_y_trues)

Epoch 0 accur: 0.530, loss: 0.283
Epoch 10 accur: 0.480, loss: 0.231
Epoch 20 accur: 0.416, loss: 0.174
Epoch 30 accur: 0.363, loss: 0.134
Epoch 40 accur: 0.322, loss: 0.107
Epoch 50 accur: 0.290, loss: 0.088
Epoch 60 accur: 0.264, loss: 0.074
Epoch 70 accur: 0.243, loss: 0.064
Epoch 80 accur: 0.225, loss: 0.055
Epoch 90 accur: 0.211, loss: 0.049


In [10]:
# Делаем пару предсказаний
emily = np.array([-7, -3]) # 128 фунтов (52.35 кг), 63 дюйма (160 см)
frank = np.array([20, 2])  # 155 pounds (63.4 кг), 68 inches (173 см)
print("Эмили: %.3f" % test(emily)) # 0.951 - Ж
print("Фрэнк: %.3f" % test(frank)) # 0.039 - М

Эмили: 0.839
Фрэнк: 0.179


In [11]:
print(LossFunction.__doc__)


    Класс содержащий классы функций ошибок,
    которые содержат в себе методы вычисляющие потерю и производные.
    
    Каждый метод классов принимает на вход два массива:
    y_true: np.array содержащий настоящии значения
    y_pred: np.array содержащий предсказанные значения
    
    Реализованны слудующие функции потерь:
    MSE: mean squeared error
    MAE: mean absolute error
    RMSE: roof mean squear error
    MAPE: mean absolute percentage error
    Huber: huber function
    LogCosh: logarithm of the hyperbolic cosine error
    Log: logarithm error
    CosineSimilarity: cosine similarity
    


In [12]:
help(LossFunction)

Help on class LossFunction in module __main__:

class LossFunction(builtins.object)
 |  Класс содержащий классы функций ошибок,
 |  которые содержат в себе методы вычисляющие потерю и производные.
 |  
 |  Каждый метод классов принимает на вход два массива:
 |  y_true: np.array содержащий настоящии значения
 |  y_pred: np.array содержащий предсказанные значения
 |  
 |  Реализованны слудующие функции потерь:
 |  MSE: mean squeared error
 |  MAE: mean absolute error
 |  RMSE: roof mean squear error
 |  MAPE: mean absolute percentage error
 |  Huber: huber function
 |  LogCosh: logarithm of the hyperbolic cosine error
 |  Log: logarithm error
 |  CosineSimilarity: cosine similarity
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined 